In [1]:
import pysam
import os
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [5]:
#read dataframe
MCVinfo = pd.read_excel('MCC_MCPV_v1.xlsx')
gtfFileGenome = 'Homo_sapiens.GRCh38.105.transcript.gtf'


# Annotate MCPV integration

In [8]:
geneDic = {}
with open(gtfFileGenome) as genome:
    rows = genome.read().rstrip().split('\n')
    for eachRow in rows:
        hasGeneName = False
        hasTrID = False
        for eachslot in eachRow.split('\t')[8].split(';'):
            if eachslot.split('"')[0] == ' transcript_id ':
                trID = eachslot.split('"')[1]
                hasTrID = True
            if eachslot.split('"')[0] == ' gene_name ':
                geneName = eachslot.split('"')[1]
                hasGeneName = True
        if not hasTrID:
            print(eachRow)
        if not hasGeneName:
            geneName = trID
        chro = eachRow.split('\t')[0]
        start = eachRow.split('\t')[3]
        end = eachRow.split('\t')[4]
        if trID not in geneDic:
            geneDic[trID] = [geneName,chro,start,end]
        else:
            print(eachRow)

In [9]:
def readData(path):
    samples = os.listdir(path)
    selectedNewContig = {}
    sampleWithFusions = 0
    re_run = []
    for sample in samples:
        if 'Sample_' in sample:
            resPath = f"{path}/{sample}/call_fusion_virus/VirusFusionPointContig.txt"
            resdicPath = f"{path}/{sample}/call_fusion_virus/filteredSelectedContig.pickle"
            with open(resdicPath,'rb') as resDicFile:
                resDic = pickle.load(resDicFile)

                selectedNewContig[sample] = resDic
            sampleWithFusions += 1
    return selectedNewContig, re_run



In [11]:
path1 = 'MCV_fusion/searcHPV_result/' #workdir
all_dic,re_run = readData(path1)

In [19]:
#add integrations
integrations = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        insList = []
        for each in all_dic['Sample_' + str(sample)]:
            chro = each.split('.')[0]
            site = each.split('.')[1]
            ins = 'chr' + chro + ':' + site
            insList.append(ins)
        integrations.append(';'.join(insList))
    else:
        integrations.append('')

In [20]:
#add number of integrations, don't add for RM samples
numIntegrations = []
i = 0
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        numIntegrations.append(len(all_dic['Sample_' + str(sample)].keys()))
    else:
        numIntegrations.append('NaN')


In [21]:
MCVinfo['Num MCV Integration'] = numIntegrations
MCVinfo['MCV Integration'] = integrations

In [25]:
#add MCV breakpoints
mcvBreakpoints = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        insListForSample = []
        for each in all_dic['Sample_' + str(sample)]:
            insList = []
            for contig in all_dic['Sample_' + str(sample)][each]:
                if contig[-1] != 'lowConfidence':
                    mcvIns = str(contig[-1])
                else:
                    mcvIns = str(contig[-2])
                insList.append(mcvIns)
            insListForSample.append(','.join(insList))
        mcvBreakpoints.append(';'.join(insListForSample))
    else:
        mcvBreakpoints.append('')



In [27]:
MCVinfo['MCV integration breakpoints on MCV genome'] = mcvBreakpoints

In [30]:
geneNum = []
geneListCol = []
for sample in MCVinfo['Sample name']:
    if 'Sample_' + str(sample) in all_dic:
        geneListSample = []
        geneNumSample = []
        for each in all_dic['Sample_' + str(sample)]:
            geneList = []
            chro = each.split('.')[0]
            site = each.split('.')[1]
            for gene in geneDic:
                if chro == geneDic[gene][1] and int(site)>= int(geneDic[gene][2]) and int(site) <= int(geneDic[gene][3]):
                    geneList.append(geneDic[gene][0])
            geneNumSample += geneList
            geneList = set(geneList)
            geneListSample.append(geneList)
    geneNum.append(len(set(geneNumSample)))
    geneListCol.append(geneListSample)
        

In [31]:
geneListColNew = []
for each in geneListCol:
    insGeneList = []
    for ins in each:
        insGeneList.append(','.join(ins))
    geneListColNew.append(';'.join(insGeneList))

In [32]:
MCVinfo['Total # Host Genes with integration'] = geneNum
MCVinfo['Host Genes with integration'] = geneListColNew

In [34]:
#Integrated in at least 1 human gene
inGeneList = []
for each in geneNum:
    inGene = False
    if each > 0:
        inGene = True
    inGeneList.append(inGene)

In [35]:
MCVinfo['Integrated in at least 1 human gene'] = inGeneList

In [36]:
#Integrated in at least 1 human cancer gene (COSMIC Teir1/2)
cancerGene = '/nfs/turbo/remillsscr/wenjingu/HPV_fusion/targeted_exom/our_pip/newVersion/analysis/uniprot_COSMIC_oncogenes_list.xlsx'
cancerGeneDf = pd.read_excel(cancerGene)


In [ ]:
cancerGeneList = []
cancerGeneDetail = []
for sample in MCVinfo['Sample name']:
    cancerGene = False
    sampleName = 'Sample_' + str(sample)
    cancerGeneNameSample = []
    if sampleName in all_dic:
        for each in all_dic[sampleName]:
            cancerGeneName = []
            chro = each.split('.')[0]
            site = each.split('.')[1]
            i = 0
            for gene in cancerGeneDf['Gene Symbol']:
                if cancerGeneDf['Tier'][i] ==1  or cancerGeneDf['Tier'][i] == 2:
                    chro2 = cancerGeneDf['Genome Location'][i].split(":")[0]
                    start = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[0]
                    end = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[1]
                    #print(chro2,start,end,cancerGeneDf['Genome Location'][i],gene)
                    if chro2!='' and start != '' and end != '':
                        if chro == chro2 and int(site) >= int(start) and int(site) <= int(end):
                            #print(cancerGene)
                            cancerGene = True
                            cancerGeneName.append(cancerGeneDf['Gene Symbol'][i])
                i += 1
            print(set(cancerGeneName))
            cancerGeneNameSample.append(",".join(set(cancerGeneName)))

    print(cancerGeneNameSample)
    cancerGeneDetail.append(';'.join(cancerGeneNameSample))                
    cancerGeneList.append(cancerGene)




In [39]:
cancerGeneList = []
cancerGeneDetail = []
for sample in MCVinfo['Sample name']:
    cancerGene = False
    sampleName = 'Sample_' + str(sample)
    cancerGeneNameSample = []
    if sampleName in all_dic:
        for each in all_dic[sampleName]:
            chro = each.split('.')[0]
            site = each.split('.')[1]
            i = 0
            for gene in cancerGeneDf['Gene Symbol']:
                if cancerGeneDf['Tier'][i] ==1  or cancerGeneDf['Tier'][i] == 2:
                    chro2 = cancerGeneDf['Genome Location'][i].split(":")[0]
                    start = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[0]
                    end = cancerGeneDf['Genome Location'][i].split(":")[1].split('-')[1]
                    #print(chro2,start,end,cancerGeneDf['Genome Location'][i],gene)
                    if chro2!='' and start != '' and end != '':
                        if chro == chro2 and int(site) >= int(start) and int(site) <= int(end):
                            #print(cancerGene)
                            cancerGene = True
                            cancerGeneNameSample.append(cancerGeneDf['Gene Symbol'][i])
                i += 1
            
    cancerGeneDetail.append(','.join(set(cancerGeneNameSample)))                
    cancerGeneList.append(cancerGene)


In [41]:
MCVinfo['Integrated in at least 1 human cancer gene (COSMIC Teir1/2)'] = cancerGeneList

In [42]:
MCVinfo['Integrated in at least 1 human cancer gene (COSMIC Teir1/2); Gene Name'] = cancerGeneDetail

In [45]:
# Annotate MCV genes
MCV_dic = {'VP2':[[465,1190]],'VP1':[[1156,2427]],'large_T':[[2503,4722],[5154,5387]],'small_T':[[4827,5387]]}

In [46]:
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        MCV_sample_list = []
        for ins in all_dic['Sample_' + str(sample)]:
            MCV_ins_list = []
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_site_list.append(MCV_gene)
                            has_gene = True
                if not has_gene:
                    MCV_site_list.append('')
                MCV_ins_list.append('+'.join(MCV_site_list))
            MCV_sample_list.append(','.join(MCV_ins_list))
        MCV_gene_list.append(';'.join(MCV_sample_list))


In [47]:
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    MCV_sample_list = []
    if str(sample) != 'nan':
        
        for ins in all_dic['Sample_' + str(sample)]:
            
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_sample_list.append(MCV_gene)
                            has_gene = True
       
               
    MCV_gene_list.append(','.join(set(MCV_sample_list)))
    


In [48]:
MCVinfo['MCV breakpoints fell into MCV gene'] = MCV_gene_list 

In [50]:
MCVinfo.to_excel('MCV_info_v2.xlsx')

# Input for link plot

In [52]:
#link plot
hpvIntegrationBed = "MCVIntegration.bed"
genomeIntegrationBed = "genomeIntegration_MCV.bed"
with open(genomeIntegrationBed,'w') as outputGenome:
    with open(hpvIntegrationBed,'w') as outputHPV:
        for sample in MCVinfo['Sample name']:
            MCV_sample_list = []
            if str(sample) != 'nan':

                for ins in all_dic['Sample_' + str(sample)]:
                    chro = ins.split('.')[0]
                    site = ins.split('.')[1]
                    for contig in all_dic['Sample_' + str(sample)][ins]:
                        if contig[-1] != 'lowConfidence':
                            MCV_site = contig[-1]
                        else:
                            MCV_site = contig[-2]
                        has_gene = False
                        for MCV_gene in MCV_dic:
                            k = 1
                            for slot in MCV_dic[MCV_gene]:
                                if MCV_site >= slot[0] and MCV_site <= slot[1]:
                                    has_gene = True
                                    if len(MCV_dic[MCV_gene]) > 1:
                                        outputHPV.write(MCV_gene + f'_{str(k)}' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n')
                                    else:
                                        outputHPV.write(MCV_gene + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n')
                                    outputGenome.write('chr' + chro + '\t' + site + '\t' + site + '\n')
                                k += 1
                        if not has_gene:
                            if MCV_site >= 0 and MCV_site <= 465:
                                outputHPV.write('intergenic_1' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            elif MCV_site >= 4722 and MCV_site <= 4827:
                                outputHPV.write('intergenic_3' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            elif MCV_site >= 2427 and MCV_site <= 2503: 
                                outputHPV.write('intergenic_2' + '\t' + str(MCV_site*40000) + '\t' + str(MCV_site* 40000) + '\n') 
                            else:
                                print(MCV_site)
                            outputGenome.write('chr' + chro + '\t' + site + '\t' + site + '\n')


In [54]:
!head MCVIntegration.bed

intergenic_1	17400000	17400000
large_T_1	164040000	164040000
VP2	30480000	30480000
VP1	96520000	96520000
large_T_1	122040000	122040000
VP1	77520000	77520000
VP1	48000000	48000000
large_T_1	162600000	162600000
VP1	72880000	72880000
large_T_1	110080000	110080000


In [55]:
!head genomeIntegration_MCV.bed

chr5	2741236	2741236
chr5	2741236	2741236
chr12	69563890	69563890
chr12	69757363	69757363
chr6	10418728	10418728
chr6	10418728	10418728
chr4	105235259	105235259
chr1	179121589	179121589
chr14	98704650	98704650
chr14	98704650	98704650


# MCV integration distribution on MCV genes

In [ ]:
#observed MCV gene count
MCV_gene_list = []
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        
        for ins in all_dic['Sample_' + str(sample)]:
            
            for contig in all_dic['Sample_' + str(sample)][ins]:
                if contig[-1] != 'lowConfidence':
                    MCV_site = contig[-1]
                else:
                    MCV_site = contig[-2]
                MCV_site_list = []
                has_gene = False
                for MCV_gene in MCV_dic:
                    for slot in MCV_dic[MCV_gene]:
                        if MCV_site >= slot[0] and MCV_site <= slot[1]:
                            MCV_gene_list.append(MCV_gene)
                            has_gene = True
  
    
print(MCV_gene_list)
countDic = dict(Counter(MCV_gene_list))
countDfMCV = pd.DataFrame.from_dict(countDic,orient='index')

In [57]:
#expect hpv integration count
totSite = 0
for each in countDic:
    totSite += countDic[each]
mcvLenDic = {}
totLen = 0
for site in MCV_dic:
    mcvLen = 0
    for each in MCV_dic[site]:
        mcvLen += int(each[1])-int(each[0])
    mcvLenDic[site] = mcvLen
    totLen += mcvLen
expectMCV = {}
for each in mcvLenDic:
    prob = mcvLenDic[each]/totLen
    expectCount = round(totSite*prob)
    expectMCV[each] = expectCount

In [58]:
expectMCVDf = pd.DataFrame.from_dict(expectMCV,orient = 'index')
countDfHPV = pd.DataFrame.merge(countDfMCV,expectMCVDf,left_index = True,right_index=True)
countDfHPV.columns = ['observed','expected']
countDfHPV.to_csv('MCV_gene.csv')

# Microhomology

In [102]:
#get blockList
def getBlock2(bam):
    sam = pysam.AlignmentFile(bam, "rb")
    blockList = {}#blockList[contig] = [start,end,cigarString]
    high_quality_contig = []
    for read in sam.fetch():
        start = read.pos
        contig = read.qname
        blocks = read.get_blocks()
        cigar = read.cigartuples
        length = read.infer_read_length()
        if read.mapping_quality >= 0:
            #if start with clip
            high_quality_contig.append(contig)
            start = 0
            end = 0
            for eachSlot in cigar:
                end += eachSlot[1]
                if not read.is_reverse:
                    if contig not in blockList:
                        blockList[contig] = [[start,end,eachSlot[0]]]
                    else:
                        blockList[contig].append([start,end,eachSlot[0]])
                else:
                    if contig not in blockList:
                        blockList[contig] = [[length-end + 1,length-start + 1,eachSlot[0]]]
                    else:
                        blockList[contig].append([length-end + 1,length-start + 1,eachSlot[0]])
                start += eachSlot[1]
        newBlockList = {}
        for contig in blockList:
            for each in  blockList[contig]:
                if each[2] == 0:
                    if contig not in newBlockList:
                        newBlockList[contig] = [[each[0],each[1]]]
                    else:
                        newBlockList[contig].append([each[0],each[1]])
                
    return newBlockList,high_quality_contig

In [103]:
#count overlap
def countOverlap(selectedContig,genomeBlock,hpvBlock):
    scoreDic = {}
    for each in selectedContig:
        score = 0
        overlap = False
        cleanEnd = False
        for slot1 in genomeBlock[each]:
            start1 = slot1[0]
            end1 = slot1[1]
            for slot2 in hpvBlock[each]:
                start2 = slot2[0]
                end2 = slot2[1]
                # overlap
                oldScore = score
                if start2 <= end1 and start1 <= end2:
                    overlap = True
                    if end1-start2 > end2-start1:
                        newScore = end2-start1 + 1
                    else:
                        newScore = end1-start2 + 1
                    if newScore > oldScore:
                        score = newScore
        #clean end
        if overlap == False:
            score = 0
            for slot1 in genomeBlock[each]:
                start1 = slot1[0]
                end1 = slot1[1]
                for slot2 in hpvBlock[each]:
                    start2 = slot2[0]
                    end2 = slot2[1]
                    #clean end
                    if start2 == end1 + 1 or start1 == end2 + 1:
                        cleanEnd = True
                        score += 0
            #gap
            if overlap == False and cleanEnd == False:
                score = 0
                for slot1 in genomeBlock[each]:
                    start1 = slot1[0]
                    end1 = slot1[1]
                    for slot2 in hpvBlock[each]:
                        start2 = slot2[0]
                        end2 = slot2[1]
                        #gap
                        oldScore = score
                        if end1 < start2 or end2 < start1:
                            if end1 < start2:
                                newScore2 = -(start2 - end1 - 1)
                            elif end2 < start1:
                                newScore2 = -(start1 - end2 - 1)
                            if newScore2 > oldScore and oldScore != 0:
                                score = newScore2
                                #print(newScore2)
                            elif oldScore == 0:
                                score = newScore2
        #filter score > 100 or <-100
#         if score < 100 and score > -100:
        scoreDic[each] = score
    return(scoreDic)

In [ ]:
scoreList = []

path = '/home/wenjingu/scratch/MCV_fusion/searcHPV_result/'
for sample in MCVinfo['Sample name']:
    if str(sample) != 'nan':
        sampleName = 'Sample_' + str(sample)
        resPath = f'{path}/{sampleName}/call_fusion_virus/'
        finalRes = f'{resPath}/filteredSelectedContig.pickle'
        with open(finalRes,'rb') as inputFile:
            resDic = pickle.load(inputFile)
        for each in resDic:
            if os.path.isdir(resPath + each):
                alignGenome = f'{resPath}/{each}/{each}.contigToGenome.sort.bam'
                alignHPV = f'{resPath}/{each}/{each}.contigToVirus.sort.bam'
                genomeBlock,high_quality_contig = getBlock2(alignGenome)
                hpvBlock,high_quality_contig2 = getBlock2(alignHPV)
                #selected Contig
                selectedContig = []
                for eachSlot in resDic[each]:
                    selectedContig.append(eachSlot[0])
                print(high_quality_contig)
                print(selectedContig)
                selectedContig = [each for each in selectedContig if each in high_quality_contig]
                selectedContig = [each for each in selectedContig if each in high_quality_contig2]
                #count overlap bp
                print(hpvBlock)
                print(selectedContig)
                scoreDic = countOverlap(selectedContig,genomeBlock,hpvBlock)
                for contig in scoreDic:
                    scoreList.append(scoreDic[contig])
                print(sample,each,scoreDic)
        #print(scoreList)

In [ ]:
with open('score.csv','w') as f:
    for each in scoreList:
        f.write(each+',')

# Information for MCPV gene link plots

In [67]:
sample = '83092/alignment/alignment.RG.indelre.mkdup.sort.bam' #use one bam to extract information about reference
test = pysam.AlignmentFile(sample, "rb")

In [70]:
chr_len = {}
for i in range(0,22):
    chr_len[int(test.references[i])] = test.lengths[i]

In [71]:
ordered_chr_len = {}
for key in sorted(chr_len):
    ordered_chr_len[key] = chr_len[key]
ordered_chr_len[test.references[23]] = test.lengths[23]
ordered_chr_len[test.references[24]] = test.lengths[24]

In [75]:
#generate bed file
bedFile = 'MCV_track.bed'
with open(bedFile,'w') as output:
    for gene in MCV_dic:
        k = 1
        for slot in MCV_dic[gene]:
            start = str((slot[0]-1)*40000)
            end = str((slot[1]-1)*40000)
            
            if len(MCV_dic[gene]) > 1:
                output.write(f'{gene}_{str(k)}\t{start}\t{end}\n')
            else:
                output.write(f'{gene}\t{start}\t{end}\n')
            k += 1
    inter1_end = str(464*40000)
    inter2_start = str(2426*40000)
    inter2_end = str(2502*40000)
    inter3_start = str(4721*40000)
    inter3_end = str(4826*40000)
    
    output.write(f'intergenic_1\t0\t{inter1_end}\n')
    output.write(f'intergenic_2\t{inter2_start}\t{inter2_end}\n')
    output.write(f'intergenic_3\t{inter3_start}\t{inter3_end}\n')

    for chro in ordered_chr_len:
        end = str(ordered_chr_len[chro])
        output.write(f'chr{chro}\t0\t{end}\n')


In [76]:
!head MCV_track.bed

VP2	18560000	47560000
VP1	46200000	97040000
large_T_1	100080000	188840000
large_T_2	206120000	215440000
small_T	193040000	215440000
intergenic_1	0	18560000
intergenic_2	97040000	100080000
intergenic_3	188840000	193040000
chr1	0	248956422
chr2	0	242193529


In [77]:
MCV_dic

{'VP2': [[465, 1190]],
 'VP1': [[1156, 2427]],
 'large_T': [[2503, 4722], [5154, 5387]],
 'small_T': [[4827, 5387]]}

In [78]:
#generate label file
labelFile = 'label.bed'
with open(labelFile,'w') as output:
    for gene in MCV_dic:
        k = 1
        for slot in MCV_dic[gene]:
            start = str((slot[0]-1)*40000/2)
            end = str((slot[1]-1)*40000/2)
            
            if len(MCV_dic[gene]) > 1:
                output.write(f'{gene}_{str(k)}\t{start}\t{end}\t{gene}\n')
            else:
                output.write(f'{gene}\t{start}\t{end}\t{gene}\n')
            k += 1
    inter1_end = str(464*40000/2)
    inter2_start = str(2426*40000/2)
    inter2_end = str(2502*40000/2)
    inter3_start = str(4721*40000/2)
    inter3_end = str(4826*40000/2)
    output.write(f'intergenic_1\t0\t{inter1_end}\tintergenic\n')
    output.write(f'intergenic_2\t{inter2_start}\t{inter2_end}\tintergenic\n')
    output.write(f'intergenic_3\t{inter3_start}\t{inter3_end}\n')

    for chro in ordered_chr_len:
        end = str(ordered_chr_len[chro]/2)
        output.write(f'chr{chro}\t0\t{end}\tchr{chro}\n')


In [79]:
!head MCV_track.bed

VP2	18560000	47560000
VP1	46200000	97040000
large_T_1	100080000	188840000
large_T_2	206120000	215440000
small_T	193040000	215440000
intergenic_1	0	18560000
intergenic_2	97040000	100080000
intergenic_3	188840000	193040000
chr1	0	248956422
chr2	0	242193529


In [80]:
!head label.bed

VP2	9280000.0	23780000.0	VP2
VP1	23100000.0	48520000.0	VP1
large_T_1	50040000.0	94420000.0	large_T
large_T_2	103060000.0	107720000.0	large_T
small_T	96520000.0	107720000.0	small_T
intergenic_1	0	9280000.0	intergenic
intergenic_2	48520000.0	50040000.0	intergenic
intergenic_3	94420000.0	96520000.0
chr1	0	124478211.0	chr1
chr2	0	121096764.5	chr2
